In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# This will reload imports before executing code, allowing you to easily change contents of custom scripts
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np

In [6]:
# #############################################################################
%matplotlib inline
import matplotlib.pyplot as plt

# Playing with Maximum Subarrays

**Goal** For a given list of +/- ints, find the subarray with a  maximum value.

**Notes** 
* the maximum value is traditionally a sum
* Complexity 
    * Brute force is $\mathcal{O}(n^3)$
    * Ways to get $\mathcal{O}(n\log n)$
    * Kadane's algo. has $\mathcal{O}(n)$
        * but it may not generalize to stay identification

**Additional notes**
* Kadane's is good but 
    * not sure if it explores the entire space
    * not clear how to generalize to min. std.
* nice to have constraints 
    * e.g. no subarrays smaller than length $m$
* nice to have early stopping criteria
* the overall score for the array is a good reference point

## Test arrays

In [7]:
test = [-2,1,-3,4,-1,2,1,-5,4]

In [15]:
a = [-13, -3, -25, -20, -3, -16, -23, -12, -5, -22, -15, -4, -7] 
aa = [-2,1,-3,4,-1,2,1,-5,4]
aaa = [2, 3, 4, 5, 7] 
aaaa = [1,2,3,4]

In [9]:
aa2 = [-2, 1, -3, 4, -1, 2, 1, -6, 4, 15, -2, 0]

In [10]:
from random import randint

In [11]:
bb = list(range(1,5)) + list(range(5,1,-1))

In [12]:
from random import shuffle

In [16]:
b = a+aa+aaa+aaaa
shuffle(b)

In [17]:
mega = np.random.randint(-100,100,1000).tolist()

## Algorithms

### Home-builts

In [18]:
def get_max_sub(l):
    sub1, sub2 = l[:-1],l[1:]
    if sum(sub1)>sum(sub2):
        return sum(sub1), sub1
    elif sum(sub2)>sum(sub1):
        return sum(sub2), sub2
    else:
        print('Equal')
        return None, None

In [19]:
get_max_sub(test)

(3, [1, -3, 4, -1, 2, 1, -5, 4])

In [20]:
class Solution(object):
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """

        max_sum = min(nums)
                
        if len(nums) == 0:
            return 0
        
        if len(nums) == 1:
            return nums[0]
        
        tot_inds = []
        for n in range(len(nums)):
            tot_inds += list(zip(list(range(n)),list(range(-(n-1),0,1))+[None]))
            
        sums = []
        for pair in tot_inds:
            sums.append(sum(nums[pair[0]:pair[1]]))
                        
        max_sum = max(sums)
        max_sum = max(max_sum, max(nums))

        return max_sum

In [21]:
s = Solution()
s.maxSubArray([-2,-1])

-1

### Kadane's algorithm

In [23]:
def max_subarray_wiki(numbers):

    """
    Find a contiguous subarray with the largest sum.
    from https://en.wikipedia.org/wiki/Maximum_subarray_problem
    
    How it works: 
    s0 = sum([0,1])
    s1 = sum([0,1,2])
    if s1 > s0: repeat 
    else s1 = sum([1,2]); repeat
    
    """

    best_sum = float('-inf') #sum([x for x in numbers if x <= 0])  # or: float('-inf')

    best_start = best_end = 0  # or: None

    current_sum = 0

    for current_end, x in enumerate(numbers):
        
        inner = f'{best_sum}, {best_start}, {best_end}'
        
        # 
        #### NOTE: potential 4eva-loop here!
        if current_sum <= 0:

            # Start a new sequence at the current element
            current_start = current_end
            current_sum = x

        else:
            # Extend the existing sequence with the current element
            current_sum += x


        if current_sum > best_sum:

            best_sum = current_sum

            best_start = current_start

            best_end = current_end + 1  # the +1 is to make 'best_end' exclusive
        
        outer =  f'{best_sum}, {best_start}, {best_end}'        
        #print('In:\t' + inner + '\tOut:\t' + outer)
        
    return best_sum, best_start, best_end

In [24]:
from sys import maxsize 
def max_subarray_gfg_linear(a): 
    """
    Function to find the maximum contiguous subarray 
    from https://www.geeksforgeeks.org/largest-sum-contiguous-subarray/
    """
    max_so_far = float('-inf')
    
    max_ending_here = 0

    size = len(a)
    
    for i in range(0, size): 
        max_ending_here = max_ending_here + a[i] 
        if (max_so_far < max_ending_here): 
            max_so_far = max_ending_here 

        if max_ending_here < 0: 
            max_ending_here = 0
    return max_so_far 

In [25]:


# Find the maximum possible sum in 
# arr[] auch that arr[m] is part of it 
def max_subarray_internal(arr, l, m, h) : 

    # Include elements on left of mid. 
    sm = 0; left_sum = -10000

    for i in range(m, l-1, -1) : 
        sm = sm + arr[i] 

        if (sm > left_sum) : 
            left_sum = sm 


    # Include elements on right of mid 
    sm = 0; right_sum = -1000
    for i in range(m + 1, h + 1) : 
        sm = sm + arr[i] 

        if (sm > right_sum) : 
            right_sum = sm 


    # Return sum of elements on left and right of mid 
    # returning only left_sum + right_sum will fail for [-2, 1] 
    return max(left_sum + right_sum, left_sum, right_sum) 


# Returns sum of maxium sum subarray in aa[l..h] 
def max_subarray_external(arr, l, h) : 

    # Base Case: Only one element 
    if (l == h) : 
        return arr[l] 

    # Find middle point 
    m = (l + h) // 2

    # Return maximum of following three possible cases 
    # a) Maximum subarray sum in left half 
    # b) Maximum subarray sum in right half 
    # c) Maximum subarray sum such that the 
    #	 subarray crosses the midpoint 
    return max(max_subarray_external(arr, l, m), 
            max_subarray_external(arr, m+1, h), 
            max_subarray_internal(arr, l, m, h)) 

def max_subarray_gfg_DC(arr):
    """
    A Divide and Conquer based program for maximum subarray sum problem 
    from https://www.geeksforgeeks.org/maximum-subarray-sum-using-divide-and-conquer-algorithm/
    """
    n = len(arr) 

    max_sum = max_subarray_external(arr, 0, n-1)
    
    return max_sum

In [26]:
for l in [a,aa,aaa]:
    print("Maximum contiguous sum is")
    print("\tWiki\t\t", max_subarray_wiki(l)[0] )
    print("\tGFG, Kadane\t", max_subarray_gfg_linear(l) )
    print("\tGFG, Div.-Conq.\t", max_subarray_gfg_DC(l) ) 

Maximum contiguous sum is
	Wiki		 -3
	GFG, Kadane	 -3
	GFG, Div.-Conq.	 -3
Maximum contiguous sum is
	Wiki		 6
	GFG, Kadane	 6
	GFG, Div.-Conq.	 6
Maximum contiguous sum is
	Wiki		 21
	GFG, Kadane	 21
	GFG, Div.-Conq.	 21


### More home-builts

In [27]:
def scoring(arr):
    
    if len(arr) >=1:
        return sum([e*1 for e in arr])
    else:
        return 0
    
score_compare = lambda current_score, best_score: current_score > best_score

def max_subarray_score(numbers):

    """
    Find a contiguous subarray with the largest sum.
    from https://en.wikipedia.org/wiki/Maximum_subarray_problem
    
    How it works: 
    s0 = sum([0,1])
    s1 = sum([0,1,2])
    if s1 > s0: repeat 
    else s1 = sum([1,2]); repeat
    
    """

    best_score = float('-inf') #sum([x for x in numbers if x <= 0])  # or: float('-inf')
    score = best_score
    best_start = best_end = 0  # or: None
    current_start = 0
    current_score = 0
    last_current_score = 0
    min_in = min(numbers)
    
    for current_end, x in enumerate(numbers):
        
        print('end', current_end)
        #inner = f'{best_score}, {best_start}, {best_end}, {score}'
        inner = f'{current_score}, {last_current_score}, {current_start}, {current_end}, {score}'
                
        #### NOTE: potential 4eva-loop here!
        #### TODO: what's the point of this hard limit? Seems only valid when starting out.
        #### When the score increases or remains constant, ok; 
        #### if it descreases, then update the limits and the score
        ### When the current_score goes < 1
        if score_compare(current_score, last_current_score)==False:
            print('\t\t\t\t\tchanged!')
            
        if score_compare(current_score, 0):
            # Extend the existing sequence with the current element
            current_score += x
            got_score = scoring(numbers[current_start:current_end+1])
            print('update current score',current_score, got_score)
        else:
            print('update start, reset current score',current_score)            
            
            # Start a new sequence at the current element
            
            current_start = current_end
            last_current_score = x
            current_score = x
        '''
        if current_end == 0:
            # Extend the existing sequence with the current element
            current_score += x        
        else:
            print('here')            
            # Start a new sequence at the current element
            current_start = current_end
            current_score = x            
        '''
        # Recording ... 
        # if current score is better than the best score, 
        #    1. update best score,
        #    2. update best limits
        if score_compare(current_score, best_score):
            print('update best limits, update best score',current_score)   

            best_score = current_score

            best_start = current_start

            best_end = current_end + 1  # the +1 is to make 'best_end' exclusive
        else:
            print('\tNo change')

        score = scoring(numbers[best_start:best_end])
        outer = f'{best_score}, {best_start}, {best_end}, {score}'
    
        print('In:\t' + inner + '\tOut:\t' + outer)
        print()
        
    return best_score, best_start, best_end

In [28]:
print(max_subarray_gfg_linear(aa))

max_subarray_score(aa)

6
end 0
					changed!
update start, reset current score 0
update best limits, update best score -2
In:	0, 0, 0, 0, -inf	Out:	-2, 0, 1, -2

end 1
					changed!
update start, reset current score -2
update best limits, update best score 1
In:	-2, -2, 0, 1, -2	Out:	1, 1, 2, 1

end 2
					changed!
update current score -2 -2
	No change
In:	1, 1, 1, 2, 1	Out:	1, 1, 2, 1

end 3
					changed!
update start, reset current score -2
update best limits, update best score 4
In:	-2, 1, 1, 3, 1	Out:	4, 3, 4, 4

end 4
					changed!
update current score 3 3
	No change
In:	4, 4, 3, 4, 4	Out:	4, 3, 4, 4

end 5
					changed!
update current score 5 5
update best limits, update best score 5
In:	3, 4, 3, 5, 4	Out:	5, 3, 6, 5

end 6
update current score 6 6
update best limits, update best score 6
In:	5, 4, 3, 6, 5	Out:	6, 3, 7, 6

end 7
update current score 1 1
	No change
In:	6, 4, 3, 7, 6	Out:	6, 3, 7, 6

end 8
					changed!
update current score 5 5
	No change
In:	1, 4, 3, 8, 6	Out:	6, 3, 7, 6



(6, 3, 7)

In [29]:
s = Solution()
s.maxSubArray(bb)

24

In [30]:
class Solution(object):
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """

        max_sum = min(nums)
        max_set = [max_sum,[]]
        
        len_nums = len(nums)
        
        if len_nums == 0:
            return 0
        
        if len_nums == 1:
            return nums[0]
        
        
        for n in range(len(nums)):

            sums = []
            sub_list_len = n
            
            inds = list(zip(list(range(sub_list_len)),list(range(-(sub_list_len-1),0,1))+[None]))

            for pair in inds:
                sublist = nums[pair[0]:pair[1]]
                #print(pair, nums, sublist)
                sum_ = sum(sublist)
                print('\t',sum_, '\t',sublist)                
                if sum_ > max_sum:
                    #print('\tmax:',sum_, sublist)
                    max_sum = sum_
                    max_set = [sum_, sublist]
                sums.append(sum_)
                
            print()#'\nsums:',sums)
        
        max_sum = max(max_sum, max(nums))
        #print('max_sum:',max_sum)
        return max_sum

In [31]:
def max_subarrays(arr):
    
    arr_len = len(arr)
    
    results = []
    subarrs = []
    new_arr = arr
    
    combined_len = 0
    
    start_stop_pairs = [[0,arr_len]]
    
    len_pairs = len(start_stop_pairs)
    
    global_indices = list(range(arr_len))    
    len_global_inds = len(global_indices)
    start, stop = global_indices[0], len_global_inds+1
    #while combined_len < arr_len:
    
    print(global_indices)
    
    #while len_pairs > 0:
    cnt = 0
    while len_global_inds > 0:
        print('\n',cnt)
        # Take the first elem of the list of pairs
        #pair = start_stop_pairs.pop(0)
        
        #start, stop = pair
        
        sub_arr = arr[start:stop]
        
        if len(sub_arr) > 1:
            result = max_subarray_wiki(sub_arr)

            # Get the local indices for the subarray, wrt that subarray!
            _, loc_start, loc_stop = result
        else:
            loc_start = 0
        
        # shift these to the global indices
        start_ = loc_start + start
        stop_  = loc_stop + start        

        print(loc_stop, start_, stop_)        
        results.append([start_, stop_]) 
        
        global_indices = [ind for ind in global_indices if (ind < start_) | (ind >= stop_)]
        
        if global_indices == []: break
        
        print(global_indices)
        diffs = [e for e,n in enumerate(list(zip(global_indices[:-1],global_indices[1:]))) if n[1]-n[0] > 1]

        if diffs == []:
            #break_ind = 0
            start, stop = global_indices[0], global_indices[-1]
        else:
            break_ind = diffs[0]
            start, stop = global_indices[0], global_indices[break_ind]+1
        
        
        print(start, stop)
       
    
        
        
        #len_pairs = len(start_stop_pairs)
        
        #len_global_inds = 0
        len_global_inds = len(global_indices)
        
        cnt+=1
        '''
        # Get the local indices for the subarray, wrt that subarray!
        _, loc_start, loc_stop = result
        
        # shift these to the global indices
        start_ += start
        stop_  += start
    
        
        if start_ == 0:
            subarr = arr[start_:stop]
        elif stop_ == arr_len:
            subarr = arr[start_:stop]            
        else:
            print('no')
        subarrs.append(subarr) 
        
        combined_len += len(arr[start:stop])            
        
        total_len += 1
        '''
    return results

In [32]:
print(max_subarray_wiki(b))
results = max_subarrays(b)

(21, 2, 10)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

 0
10 2 10
[0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
0 2

 1
2 1 2
[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
0 1

 2
2 0 2
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
10 30

 3
11 19 21
[10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
10 19

 4
6 15 16
[10, 11, 12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
10 15

 5
1 10 11
[11, 12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
11 15

 6
1 11 12
[12, 13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
12 15

 7
1 12 13
[13, 14, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
13 15

 8
2 14 15
[13, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
13 14

 9
2 13 15
[16, 17, 18, 21, 22, 23, 24, 25, 26, 27,

In [33]:
def max_subarray_wiki1(numbers):

    """Find the largest sum of any contiguous subarray."""

    best_sum = 0  # or: float('-inf')

    current_sum = 0

    for x in numbers:

        current_sum = max(0, current_sum + x)

        best_sum = max(best_sum, current_sum)
        print(x, best_sum, current_sum)
        
    return best_sum

In [34]:
max_subarray_wiki1(aa2)

-2 0 0
1 1 1
-3 1 0
4 4 4
-1 4 3
2 5 5
1 6 6
-6 6 0
4 6 4
15 19 19
-2 19 17
0 19 17


19

In [35]:
max_subarray_wiki(aa2)

(19, 8, 10)

In [36]:
s.maxSubArray(aa2)

19

#### Brute-force

SHows all subarrays and their scores

In [37]:
#class Solution(object):
def maxSubArray2(nums, verbose=False):
    """
    :type nums: List[int]
    :rtype: int
    """

    max_sum = min(nums)
    max_set = [max_sum,[]]

    len_nums = len(nums)

    if len_nums == 0:
        return 0

    if len_nums == 1:
        return nums[0]


    for n in range(len(nums)):

        sums = []
        sub_list_len = n

        inds = list(zip(list(range(sub_list_len)),list(range(-(sub_list_len-1),0,1))+[None]))
        
        # All the subsets
        
        for pair in inds:
            sublist = nums[pair[0]:pair[1]]
            #print(pair, nums, sublist)
            sum_ = sum(sublist)
            max_sum_str = ''     
            if sum_ > max_sum:
                #print('\tmax:',sum_, sublist)
                max_sum = sum_
                max_set = [sum_, sublist]
                max_sum_str = str(max_sum)
                new_pair = pair
            if verbose: print('\t',sum_, '\t',pair,'\t', sublist, '\t'+max_sum_str) 
            
            sums.append(sum_)

        if verbose: print()#'\nsums:',sums)

    max_sum = max(max_sum, max(nums))
    #print('max_sum:',max_sum)
    return max_sum

In [39]:
maxSubArray2(aa2, True)


	 13 	 (0, None) 	 [-2, 1, -3, 4, -1, 2, 1, -6, 4, 15, -2, 0] 	13

	 13 	 (0, -1) 	 [-2, 1, -3, 4, -1, 2, 1, -6, 4, 15, -2] 	
	 15 	 (1, None) 	 [1, -3, 4, -1, 2, 1, -6, 4, 15, -2, 0] 	15

	 15 	 (0, -2) 	 [-2, 1, -3, 4, -1, 2, 1, -6, 4, 15] 	
	 15 	 (1, -1) 	 [1, -3, 4, -1, 2, 1, -6, 4, 15, -2] 	
	 14 	 (2, None) 	 [-3, 4, -1, 2, 1, -6, 4, 15, -2, 0] 	

	 0 	 (0, -3) 	 [-2, 1, -3, 4, -1, 2, 1, -6, 4] 	
	 17 	 (1, -2) 	 [1, -3, 4, -1, 2, 1, -6, 4, 15] 	17
	 14 	 (2, -1) 	 [-3, 4, -1, 2, 1, -6, 4, 15, -2] 	
	 17 	 (3, None) 	 [4, -1, 2, 1, -6, 4, 15, -2, 0] 	

	 -4 	 (0, -4) 	 [-2, 1, -3, 4, -1, 2, 1, -6] 	
	 2 	 (1, -3) 	 [1, -3, 4, -1, 2, 1, -6, 4] 	
	 16 	 (2, -2) 	 [-3, 4, -1, 2, 1, -6, 4, 15] 	
	 17 	 (3, -1) 	 [4, -1, 2, 1, -6, 4, 15, -2] 	
	 13 	 (4, None) 	 [-1, 2, 1, -6, 4, 15, -2, 0] 	

	 2 	 (0, -5) 	 [-2, 1, -3, 4, -1, 2, 1] 	
	 -2 	 (1, -4) 	 [1, -3, 4, -1, 2, 1, -6] 	
	 1 	 (2, -3) 	 [-3, 4, -1, 2, 1, -6, 4] 	
	 19 	 (3, -2) 	 [4, -1, 2, 1, -6, 4, 15] 	19
	 13 	 (4, -1) 	

19

In [41]:
print(sum(mega))
print(maxSubArray2(mega))

1841
3644


In [42]:
max_subarray_gfg_linear(mega)

3644

# splitting brute force method

**Algorithm**
1. compaute score for size-$N$ array
2. Split array into 2 subarray, each with $N-1$ elements
    * `[a:b]` $\to$ `[a:b-1],[a+1:b]`
3. Choose the child with the better score
4. Repeat until some stopping criterion is met

**Notes**
* Not guaranteed to find the global max
* for large datasets, it will stop too early, removing the subarray from further calculation

**TODOs**
* side-stepping? from a given node with decreasing prospects, go to another node on the same level with better prospects, namely that it's children are increasing from it's values.

In [685]:
get_pairs_list = lambda max_len: list(zip(list(range(max_len)), list(range(-(max_len-1),0,1))+[None]))

get_pairs_from_pair = lambda pair: \
    [(pair[0],pair[1]-1), (pair[0]+1,pair[1])] if pair[1] != None \
    else [(pair[0],-1), (pair[0]+1,pair[1])]

#class Solution(object):
def maxSubArray_pairs(nums):
    """
    :type nums: List[int]
    :rtype: int
    """

    max_sum = float('-inf')
    max_set = [max_sum,[]]

    len_nums = len(nums)

    if len_nums == 0:
        return 0

    if len_nums == 1:
        return nums[0]

    new_pair = (0,None)

    sums = []    
    
    for n in range(len_nums):
        
        # Only two subsets 
        p1,p2 =  get_pairs_from_pair(new_pair)
        #print(p1,p2)
            
        sublist1 = nums[p1[0]:p1[1]]
        sublist2 = nums[p2[0]:p2[1]]
        
        #print(pair, nums, sublist)
        sum1_ = sum(sublist1)
        sum2_ = sum(sublist2)

        #print(sum1_, sublist1,'\n', sum2_, sublist2)
        if sum1_ >= sum2_:
            new_pair = p1
            sublist = sublist1
            sum_ = sum1_
        else:
            new_pair = p2
            sublist = sublist2
            sum_ = sum2_

        max_sum_str = ''     
        
        
        
        if sum_ > max_sum:
            #print('\tmax:',sum_, sublist)
            max_sum = sum_
            max_set = [sum_, sublist]
            max_sum_str = str(max_sum)
            print(len(sublist),max_sum)
            print('\t',sum_,'\t', new_pair, '\t'+max_sum_str)
        else:

            #print(sum1_, sublist1,'\n', sum2_, sublist2)
            if sum1_ == sum_:
                new_pair = p1
                sublist = sublist1
                sum_ = sum1_
            else:
                new_pair = p2
                sublist = sublist2
                sum_ = sum2_            
                #break
            print('Switch!\t',sum_,'\t', new_pair, '\t'+max_sum_str)
        #print('\t',sum_,'\t', sublist, '\t'+max_sum_str) 
        #print('\t',sum_,'\t', new_pair, '\t'+max_sum_str)
        
        sums.append(sum_)

        #print()#'\nsums:',sums)

    max_sum = max(max_sum, max(nums))
    #print('max_sum:',max_sum)
    return max_sum

In [686]:
maxSubArray_pairs(mega)

999 -1706
	 -1706 	 (0, -1) 	-1706
Switch!	 -1719 	 (0, -2) 	
Switch!	 -1764 	 (1, -2) 	
Switch!	 -1828 	 (1, -3) 	
Switch!	 -1864 	 (1, -4) 	
Switch!	 -1852 	 (1, -5) 	
Switch!	 -1814 	 (1, -6) 	
Switch!	 -1850 	 (1, -7) 	
Switch!	 -1804 	 (1, -8) 	
Switch!	 -1875 	 (1, -9) 	
Switch!	 -1934 	 (1, -10) 	
Switch!	 -2006 	 (2, -10) 	
Switch!	 -1936 	 (3, -10) 	
Switch!	 -1895 	 (4, -10) 	
Switch!	 -1965 	 (5, -10) 	
Switch!	 -1882 	 (6, -10) 	
Switch!	 -1894 	 (7, -10) 	
Switch!	 -1873 	 (8, -10) 	
Switch!	 -1960 	 (8, -11) 	
Switch!	 -2032 	 (8, -12) 	
Switch!	 -2039 	 (8, -13) 	
Switch!	 -1981 	 (8, -14) 	
Switch!	 -2028 	 (8, -15) 	
Switch!	 -2091 	 (8, -16) 	
Switch!	 -2155 	 (8, -17) 	
Switch!	 -2180 	 (8, -18) 	
Switch!	 -2187 	 (8, -19) 	
Switch!	 -2258 	 (8, -20) 	
Switch!	 -2295 	 (8, -21) 	
Switch!	 -2214 	 (8, -22) 	
Switch!	 -2245 	 (8, -23) 	
Switch!	 -2235 	 (8, -24) 	
Switch!	 -2225 	 (8, -25) 	
Switch!	 -2257 	 (8, -26) 	
Switch!	 -2300 	 (8, -27) 	
Switch!	 -2325 	 (8, -

Switch!	 -1389 	 (95, -307) 	
Switch!	 -1408 	 (95, -308) 	
Switch!	 -1466 	 (95, -309) 	
Switch!	 -1392 	 (95, -310) 	
Switch!	 -1355 	 (95, -311) 	
Switch!	 -1303 	 (95, -312) 	
Switch!	 -1279 	 (95, -313) 	
Switch!	 -1212 	 (95, -314) 	
Switch!	 -1227 	 (95, -315) 	
Switch!	 -1209 	 (95, -316) 	
Switch!	 -1262 	 (95, -317) 	
Switch!	 -1256 	 (95, -318) 	
Switch!	 -1248 	 (95, -319) 	
Switch!	 -1168 	 (95, -320) 	
Switch!	 -1192 	 (95, -321) 	
Switch!	 -1098 	 (95, -322) 	
Switch!	 -1102 	 (95, -323) 	
Switch!	 -1158 	 (95, -324) 	
Switch!	 -1189 	 (95, -325) 	
Switch!	 -1213 	 (95, -326) 	
Switch!	 -1158 	 (95, -327) 	
Switch!	 -1232 	 (95, -328) 	
Switch!	 -1198 	 (95, -329) 	
Switch!	 -1230 	 (95, -330) 	
Switch!	 -1245 	 (95, -331) 	
Switch!	 -1226 	 (95, -332) 	
Switch!	 -1203 	 (95, -333) 	
Switch!	 -1251 	 (95, -334) 	
Switch!	 -1228 	 (95, -335) 	
Switch!	 -1157 	 (95, -336) 	
Switch!	 -1155 	 (95, -337) 	
Switch!	 -1223 	 (95, -338) 	
Switch!	 -1192 	 (95, -339) 	
Switch!	 -

Switch!	 846 	 (95, -734) 	
Switch!	 790 	 (95, -735) 	
Switch!	 706 	 (95, -736) 	
Switch!	 738 	 (95, -737) 	
Switch!	 832 	 (95, -738) 	
Switch!	 768 	 (95, -739) 	
Switch!	 688 	 (95, -740) 	
Switch!	 768 	 (95, -741) 	
Switch!	 837 	 (95, -742) 	
Switch!	 761 	 (95, -743) 	
Switch!	 840 	 (95, -744) 	
Switch!	 884 	 (95, -745) 	
Switch!	 953 	 (95, -746) 	
Switch!	 998 	 (95, -747) 	
Switch!	 957 	 (95, -748) 	
Switch!	 917 	 (95, -749) 	
Switch!	 848 	 (95, -750) 	
Switch!	 803 	 (95, -751) 	
Switch!	 832 	 (95, -752) 	
Switch!	 902 	 (95, -753) 	
Switch!	 845 	 (95, -754) 	
Switch!	 938 	 (95, -755) 	
Switch!	 866 	 (95, -756) 	
Switch!	 871 	 (95, -757) 	
Switch!	 814 	 (95, -758) 	
Switch!	 788 	 (95, -759) 	
Switch!	 802 	 (95, -760) 	
Switch!	 706 	 (95, -761) 	
Switch!	 664 	 (95, -762) 	
Switch!	 611 	 (95, -763) 	
Switch!	 589 	 (95, -764) 	
Switch!	 647 	 (95, -765) 	
Switch!	 676 	 (95, -766) 	
Switch!	 606 	 (95, -767) 	
Switch!	 630 	 (95, -768) 	
Switch!	 632 	 (95, 

1065

In [668]:
max_subarray_wiki(mega)

(1305, 109, 171)